In [ ]:
# Standard library imports
import os
import random
import warnings

# Third-party imports
import tensorflow as tf
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model

# Setting up matplotlib to work interactively in a Jupyter environment
%matplotlib inline

# Setting the seed for reproducibility
seed_value = 42
random.seed(seed_value)  # For Python's built-in random module
tf.random.set_seed(seed_value)  # For TensorFlow

# Don't show warning messages
warnings.filterwarnings('ignore')


In [2]:
def custom_loss(y_true, y_pred):
    # Compute cross entropy loss
    cross_entropy_loss = K.binary_crossentropy(y_true, y_pred)
    
    # Compute IoU loss
    intersection = K.sum(K.abs(y_true * y_pred))
    union = K.sum(y_true) + K.sum(y_pred) - intersection
    iou_loss = 1 - (intersection + 1) / (union + 1)  # Adding 1 to avoid division by zero
    
   
    # Combine the losses
    loss = cross_entropy_loss + .5* iou_loss   # Jaccard loss with binary cross entropy
    
    return loss

In [ ]:
from unet_model import UNET

In [ ]:
# Create empty arrays
IMG_HEIGHT = 1024
per_image_norm = False

In [ ]:
base_ptch_1024 = UNET(IMG_HEIGHT)
base_ptch_1024.exec_()

In [ ]:
import tensorflow as tf

def train_model(base_ptch_1024, train_gen, val_gen):
    # Define the ModelCheckpoint callback to save only the best model based on validation loss
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath='best_model.h5',  # This is the file path where the best model will be saved
        monitor='val_loss',  # Monitor validation loss to determine the best model
        verbose=1,
        save_best_only=True,  # Save only the model that has the best 'val_loss'
        mode='min'  # The monitoring metric (here, 'val_loss') is expected to minimize
    )

    # Define other callbacks as needed, e.g., for early stopping or logging with TensorBoard
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs'),
        model_checkpoint_callback  # Include the ModelCheckpoint callback here
    ]

    # Fit the model with your training and validation data
    results = base_ptch_1024.model.fit(
        train_gen,
        epochs=70,
        steps_per_epoch=int(len(t_list)/btch_sz),  # Explicitly set this   # changed train_list 
        validation_data=val_gen,
        validation_steps=int(len(v_list)/btch_sz),  # And this
        callbacks=callbacks
    )



### Sample Usage

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:

      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    

In [ ]:
# Create empty arrays
IMG_HEIGHT = 1024
IMG_WIDTH = 1024
IMG_CHANNELS = 1
per_image_norm = False

# Add these constants for brightness, saturation, and hue adjustments
BRIGHTNESS = 0.2
SATURATION = 0.3
HUE = 0.25

# Define Paths
data_path = r'< full_train_data_path >' 
lab_path =  r'< full_test_data_path >'
part_data_path = r'< part_train_data >'   
part_lab_path = r'< part_test_data >'
part_train_list = os.listdir(part_data_path)
test_list = os.listdir(data_path)

get_mean_and_std(data_path)
train_gen, val_gen, test_gen_large = preprocess_and_load(data_path, part_data_path, part_lab_path, lab_path, train_list, test_list)

train_model(base_ptch_1024, train_gen, val_gen)